In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ten

## Titanic 다시 해보기 1
1. Pclass 바꿔보기
2. Title 분류 바꿔보기

In [2]:
# Title, Age

In [3]:
train_df = pd.read_csv("./data/titanic/train.csv")
train_df
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train_df["Title"] = train_df["Name"].str.extract("([A-Za-z]+)\.")
train_df["Title"].value_counts()
train_df.groupby("Title")["Age"].mean()

Title
Capt        70.000000
Col         58.000000
Countess    33.000000
Don         40.000000
Dr          42.000000
Jonkheer    38.000000
Lady        48.000000
Major       48.500000
Master       4.574167
Miss        21.773973
Mlle        24.000000
Mme         24.000000
Mr          32.368090
Mrs         35.898148
Ms          28.000000
Rev         43.166667
Sir         49.000000
Name: Age, dtype: float64

In [5]:
title_mapping_dict = {"Mr" : 0, "Don" : 0, "Sir" : 0,
                      "Miss" : 1, "Ms" : 1, "Mlle" : 1, "Mme": 1,
                      "Mrs" : 2, "Lady" : 2, "Countess" : 2,
                      "Capt" : 3, "Col" : 3, "Dr" : 3, "Jonkheer" : 3, "Major" : 3, "Master" : 3, "Rev" : 3}
train_df["Title"] = train_df["Title"].map(title_mapping_dict)
train_df["Title"].isnull().sum()
train_df["Title"].value_counts()

0    519
1    186
2    127
3     59
Name: Title, dtype: int64

In [6]:
age_mean = train_df.groupby("Title")["Age"].mean()              
age_mean

a = train_df[train_df["Title"]==0]["Age"].fillna(age_mean[0])   
b = train_df[train_df["Title"]==1]["Age"].fillna(age_mean[1])
c = train_df[train_df["Title"]==2]["Age"].fillna(age_mean[2])
d = train_df[train_df["Title"]==3]["Age"].fillna(age_mean[3])

result = pd.concat([a,b,c,d])
result                                                         
result.sort_index()

train_df["Age"] = result.sort_index()
train_df
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
dtype: int64

In [7]:
train_df.loc[train_df["Age"] <= 20, "Age"] = 0          # Age 20 이하인 행들, 그 행의 Age를 0으로.
train_df.loc[(train_df["Age"] > 20) & (train_df["Age"] <= 40), "Age"] = 1
train_df.loc[(train_df["Age"] > 40) & (train_df["Age"] <= 60), "Age"] = 2
train_df.loc[train_df["Age"] > 60, "Age"] = 3
train_df["Age"].value_counts()

1.0    557
0.0    184
2.0    128
3.0     22
Name: Age, dtype: int64

In [8]:
# pclass 바꿔주기

pclass_mapping_dict = {1 : 3, 2 : 2, 3 : 1}
train_df["Pclass"] = train_df["Pclass"].map(pclass_mapping_dict)
train_df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Title
PassengerId,1.000000,-0.005007,0.035144,0.025547,-0.057527,-0.001652,0.012658,-0.004496
Survived,-0.005007,1.000000,0.338481,-0.066813,-0.035322,0.081629,0.257307,0.439436
Pclass,0.035144,0.338481,1.000000,0.293328,-0.083081,-0.018443,0.549500,0.141142
Age,0.025547,-0.066813,0.293328,1.000000,-0.215443,-0.154772,0.071467,-0.123395
SibSp,-0.057527,-0.035322,-0.083081,-0.215443,1.000000,0.414838,0.159651,0.298440
Parch,-0.001652,0.081629,-0.018443,-0.154772,0.414838,1.000000,0.216225,0.358599
Fare,0.012658,0.257307,0.549500,0.071467,0.159651,0.216225,1.000000,0.141765
Title,-0.004496,0.439436,0.141142,-0.123395,0.298440,0.358599,0.141765,1.000000


In [9]:
# sex
sex_mapping_dict = {"male" : 0, "female" : 1}
train_df["Sex"] = train_df["Sex"].map(sex_mapping_dict)
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,1,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,3,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,1,"Heikkinen, Miss. Laina",1,1.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1.0,1,0,113803,53.1000,C123,S,2
4,5,0,1,"Allen, Mr. William Henry",0,1.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,1.0,0,0,211536,13.0000,NaN,S,3
887,888,1,3,"Graham, Miss. Margaret Edith",1,0.0,0,0,112053,30.0000,B42,S,1
888,889,0,1,"Johnston, Miss. Catherine Helen ""Carrie""",1,1.0,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,3,"Behr, Mr. Karl Howell",0,1.0,0,0,111369,30.0000,C148,C,0


In [10]:
# Family 
train_df["Family"] = train_df["SibSp"] + train_df["Parch"]
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family
0,1,0,1,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,3,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1.0,1,0,PC 17599,71.2833,C85,C,2,1
2,3,1,1,"Heikkinen, Miss. Laina",1,1.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0
3,4,1,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1.0,1,0,113803,53.1000,C123,S,2,1
4,5,0,1,"Allen, Mr. William Henry",0,1.0,0,0,373450,8.0500,NaN,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,1.0,0,0,211536,13.0000,NaN,S,3,0
887,888,1,3,"Graham, Miss. Margaret Edith",1,0.0,0,0,112053,30.0000,B42,S,1,0
888,889,0,1,"Johnston, Miss. Catherine Helen ""Carrie""",1,1.0,1,2,W./C. 6607,23.4500,NaN,S,1,3
889,890,1,3,"Behr, Mr. Karl Howell",0,1.0,0,0,111369,30.0000,C148,C,0,0


In [11]:
# fare binning
train_df.loc[train_df["Fare"] <= 100, "Fare"] = 0 
train_df.loc[(train_df["Fare"] > 100) & (train_df["Fare"] <= 200), "Fare"] = 1
train_df.loc[(train_df["Fare"] > 200) & (train_df["Fare"] <= 300), "Fare"] = 2
train_df.loc[train_df["Fare"] > 300, "Fare"] = 3
train_df["Fare"].value_counts()

0.0    838
1.0     33
2.0     17
3.0      3
Name: Fare, dtype: int64

In [12]:
# embarked
train_df["Embarked"].fillna("S", inplace = True)
embarked_mapping_dict = {"S" : 0, "Q" : 1, "C" : 2}
train_df["Embarked"] = train_df["Embarked"].map(embarked_mapping_dict)

In [13]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family
0,1,0,1,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,0.0,NaN,0,0,1
1,2,1,3,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1.0,1,0,PC 17599,0.0,C85,2,2,1
2,3,1,1,"Heikkinen, Miss. Laina",1,1.0,0,0,STON/O2. 3101282,0.0,NaN,0,1,0
3,4,1,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1.0,1,0,113803,0.0,C123,0,2,1
4,5,0,1,"Allen, Mr. William Henry",0,1.0,0,0,373450,0.0,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,1.0,0,0,211536,0.0,NaN,0,3,0
887,888,1,3,"Graham, Miss. Margaret Edith",1,0.0,0,0,112053,0.0,B42,0,1,0
888,889,0,1,"Johnston, Miss. Catherine Helen ""Carrie""",1,1.0,1,2,W./C. 6607,0.0,NaN,0,1,3
889,890,1,3,"Behr, Mr. Karl Howell",0,1.0,0,0,111369,0.0,C148,2,0,0


In [14]:
train_df.drop("Name", axis = 1, inplace = True)
train_df.drop("SibSp", axis = 1, inplace = True)
train_df.drop("Parch", axis = 1, inplace = True)
train_df.drop("Ticket", axis = 1, inplace = True)
train_df.drop("Cabin", axis = 1, inplace = True)

In [15]:
train_df
train_df.info()
train_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
Embarked       891 non-null int64
Title          891 non-null int64
Family         891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.8 KB


PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
Fare           0
Embarked       0
Title          0
Family         0
dtype: int64

### 머신러닝 진행

In [16]:
train_df.drop("PassengerId", axis = 1, inplace = True)
train_df

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Family
0,0,1,0,1.0,0.0,0,0,1
1,1,3,1,1.0,0.0,2,2,1
2,1,1,1,1.0,0.0,0,1,0
3,1,3,1,1.0,0.0,0,2,1
4,0,1,0,1.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...
886,0,2,0,1.0,0.0,0,3,0
887,1,3,1,0.0,0.0,0,1,0
888,0,1,1,1.0,0.0,0,1,3
889,1,3,0,1.0,0.0,2,0,0


In [17]:
# data 준비

train_df.shape

train_num = int(train_df.shape[0] * 0.8)        
test_num = train_df.shape[0] - train_num

train_x_data = train_df.drop("Survived", axis = 1, inplace = False)[:train_num].values
train_y_data = train_df["Survived"][:train_num].values.reshape([-1, 1])

test_x_data = train_df.drop("Survived", axis = 1, inplace = False)[train_num:].values
test_y_data = train_df["Survived"][train_num:].values.reshape([-1, 1])

In [18]:
# 다음 단계

train_x_data.shape
train_y_data.shape
X = tf.placeholder(shape = [None, 7], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 1], dtype = tf.float32)

W = tf.Variable(tf.random_normal([7,1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")

logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit,
                                                              labels = Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(30000):
    _, cost_val = sess.run([train, cost],
                          feed_dict = {X : train_x_data,
                                      Y: train_y_data})
    if step % 3000 == 0:
        print("cost 값은 : {}".format(cost_val))
        
predict = tf.cast(H > 0.5, dtype = tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype= tf.float32))
print("정확도 : {}".format(sess.run(accuracy, feed_dict = {X : test_x_data,
                                                       Y : test_y_data})))

cost 값은 : 1.6307957172393799
cost 값은 : 0.4747092127799988
cost 값은 : 0.45421043038368225
cost 값은 : 0.44487762451171875
cost 값은 : 0.44019338488578796
cost 값은 : 0.43781888484954834
cost 값은 : 0.4366109073162079
cost 값은 : 0.4359916150569916
cost 값은 : 0.43567073345184326
cost 값은 : 0.4355029761791229
정확도 : 0.8379888534545898


# test.csv 파일 제출용

In [19]:
test_df = pd.read_csv("./data/titanic/test.csv")
test_df.head()
test_df.shape
test_df.info()
test_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [20]:
test_df["Title"] = test_df["Name"].str.extract("([A-Za-z]+)\.")
test_df["Title"].value_counts()
test_df.groupby("Title")["Age"].mean()

Title
Col       50.000000
Dona      39.000000
Dr        53.000000
Master     7.406471
Miss      21.774844
Mr        32.000000
Mrs       38.903226
Ms              NaN
Rev       35.500000
Name: Age, dtype: float64

In [21]:
title_mapping_dict = {"Mr" : 0, "Don" : 0, "Sir" : 0,
                      "Miss" : 1, "Ms" : 1, "Mlle" : 1, "Mme": 1,
                      "Mrs" : 2, "Lady" : 2, "Countess" : 2, "Dona" : 2,
                      "Capt" : 3, "Col" : 3, "Dr" : 3, "Jonkheer" : 3, "Major" : 3, "Master" : 3, "Rev" : 3}
test_df["Title"] = test_df["Title"].map(title_mapping_dict)
test_df["Title"].isnull().sum()
test_df["Title"].value_counts()

0    240
1     79
2     73
3     26
Name: Title, dtype: int64

In [22]:
age_mean = test_df.groupby("Title")["Age"].mean()              
age_mean

a = test_df[test_df["Title"]==0]["Age"].fillna(age_mean[0])   
b = test_df[test_df["Title"]==1]["Age"].fillna(age_mean[1])
c = test_df[test_df["Title"]==2]["Age"].fillna(age_mean[2])
d = test_df[test_df["Title"]==3]["Age"].fillna(age_mean[3])

result = pd.concat([a,b,c,d])
result                                                         
result.sort_index()

test_df["Age"] = result.sort_index()
test_df
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
dtype: int64

In [23]:
fare_mean = test_df["Fare"].mean()
test_df["Fare"] = test_df["Fare"].fillna(fare_mean)

test_df.isnull().sum()
print(test_df["Fare"].min())
print(test_df["Fare"].max())

0.0
512.3292


In [24]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Title            0
dtype: int64

In [25]:
test_df.loc[test_df["Age"] <= 20, "Age"] = 0          # Age 20 이하인 행들, 그 행의 Age를 0으로.
test_df.loc[(test_df["Age"] > 20) & (test_df["Age"] <= 40), "Age"] = 1
test_df.loc[(test_df["Age"] > 40) & (test_df["Age"] <= 60), "Age"] = 2
test_df.loc[test_df["Age"] > 60, "Age"] = 3
test_df["Age"].value_counts()

1.0    268
0.0     73
2.0     66
3.0     11
Name: Age, dtype: int64

In [26]:
pclass_mapping_dict = {1 : 3, 2 : 2, 3 : 1}
test_df["Pclass"] = test_df["Pclass"].map(pclass_mapping_dict)

In [27]:
sex_mapping_dict = {"male" : 0, "female" : 1}
test_df["Sex"] = test_df["Sex"].map(sex_mapping_dict)
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,1,"Kelly, Mr. James",0,1.0,0,0,330911,7.8292,NaN,Q,0
1,893,1,"Wilkes, Mrs. James (Ellen Needs)",1,2.0,1,0,363272,7.0000,NaN,S,2
2,894,2,"Myles, Mr. Thomas Francis",0,3.0,0,0,240276,9.6875,NaN,Q,0
3,895,1,"Wirz, Mr. Albert",0,1.0,0,0,315154,8.6625,NaN,S,0
4,896,1,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1.0,1,1,3101298,12.2875,NaN,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,1,"Spector, Mr. Woolf",0,1.0,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,3,"Oliva y Ocana, Dona. Fermina",1,1.0,0,0,PC 17758,108.9000,C105,C,2
415,1307,1,"Saether, Mr. Simon Sivertsen",0,1.0,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,1,"Ware, Mr. Frederick",0,1.0,0,0,359309,8.0500,NaN,S,0


In [28]:
test_df["Family"] = test_df["SibSp"] + test_df["Parch"]
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family
0,892,1,"Kelly, Mr. James",0,1.0,0,0,330911,7.8292,NaN,Q,0,0
1,893,1,"Wilkes, Mrs. James (Ellen Needs)",1,2.0,1,0,363272,7.0000,NaN,S,2,1
2,894,2,"Myles, Mr. Thomas Francis",0,3.0,0,0,240276,9.6875,NaN,Q,0,0
3,895,1,"Wirz, Mr. Albert",0,1.0,0,0,315154,8.6625,NaN,S,0,0
4,896,1,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1.0,1,1,3101298,12.2875,NaN,S,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,1,"Spector, Mr. Woolf",0,1.0,0,0,A.5. 3236,8.0500,NaN,S,0,0
414,1306,3,"Oliva y Ocana, Dona. Fermina",1,1.0,0,0,PC 17758,108.9000,C105,C,2,0
415,1307,1,"Saether, Mr. Simon Sivertsen",0,1.0,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,0
416,1308,1,"Ware, Mr. Frederick",0,1.0,0,0,359309,8.0500,NaN,S,0,0


In [29]:
test_df.loc[test_df["Fare"] <= 100, "Fare"] = 0 
test_df.loc[(test_df["Fare"] > 100) & (test_df["Fare"] <= 200), "Fare"] = 1
test_df.loc[(test_df["Fare"] > 200) & (test_df["Fare"] <= 300), "Fare"] = 2
test_df.loc[test_df["Fare"] > 300, "Fare"] = 3
test_df["Fare"].value_counts()

0.0    387
2.0     17
1.0     13
3.0      1
Name: Fare, dtype: int64

In [30]:
test_df["Embarked"].fillna("S", inplace = True)
embarked_mapping_dict = {"S" : 0, "Q" : 1, "C" : 2}
test_df["Embarked"] = test_df["Embarked"].map(embarked_mapping_dict)

In [31]:
test_df.drop("Name", axis = 1, inplace = True)
test_df.drop("SibSp", axis = 1, inplace = True)
test_df.drop("Parch", axis = 1, inplace = True)
test_df.drop("Ticket", axis = 1, inplace = True)
test_df.drop("Cabin", axis = 1, inplace = True)

In [32]:
predict_x_data = test_df.drop("PassengerId", axis = 1, inplace = False).values
predict_x_data

predict_test = tf.cast(H > 0.5, dtype = tf.float32)
result = sess.run(predict_test, feed_dict = {X : predict_x_data})
result = pd.DataFrame(result)
result

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
413,0.0
414,1.0
415,0.0
416,0.0


In [33]:
submit_df = pd.read_csv("./data/titanic/gender_submission.csv")
display(submit_df)
submit_df["Survived"] = result
submit_df = pd.DataFrame(submit_df, dtype = 'int32')

submit_df.to_csv("./data/titanic/submission_2.csv",
                 sep = ",",
                 header = True,
                 index = False)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
